In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold,cross_val_score,StratifiedShuffleSplit
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from collections import Counter
import seaborn as sns

In [0]:
data = pd.read_csv("drive/My Drive/Thesis/Wildfire.csv")
X = data.loc[:,['NDVI','LST','Burned_Area']].values.astype('float32')
nor_X = preprocessing.normalize(X)
Y = data.loc[:,['Class']].values.astype('int')
labelencoder_y_1 = LabelEncoder()
Y = labelencoder_y_1.fit_transform(Y)

In [0]:
np.random.seed(1080)
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.20)

In [0]:
kfold = StratifiedShuffleSplit(n_splits=10, test_size=0.15, random_state=12)
training_accuracy = []
testing_accuracy = []
epochs=10
for train, test in kfold.split(X_train, y_train):

  clf = MLPClassifier(solver='sgd',learning_rate='adaptive',momentum=0.9, activation='relu',alpha=5, batch_size='auto',verbose=True,n_iter_no_change = epochs)
  clf.fit(X[train],Y[train])
  model = clf
  
  #training accuracy
  y_tr_pred = clf.predict(X[train])
  results_tr = cross_val_score(model, X[train] ,y_tr_pred, cv = kfold,verbose=1)
  training_accuracy.append(results_tr.max()*100.0)
  
  #testing Accuracy
  y_te_pred = clf.predict(X[test])
  results = cross_val_score(model, X[test] ,y_te_pred, cv = kfold,verbose=1)
  testing_accuracy.append(results.max()*100.0)
  
  print("----------------------------------")
  print(confusion_matrix(Y[test],y_te_pred))
  print("----------------------------------")
  print(classification_report(Y[test],y_te_pred))
  
  print("Training Accuracy (Shuffle Split) : %.3f%% (%.3f%%)" % (results_tr.max()*100.0, results_tr.std()*100.0))
  print("Prediction Accuracy (Shuffle Split) : %.3f%% (%.3f%%)" % (results.max()*100.0, results.std()*100.0))

In [0]:
 print(confusion_matrix(y_test,y_te_pred))

In [0]:
print(classification_report(y_test,y_te_pred))

In [0]:
iterations = list(range(epochs))
plt.plot(iterations, training_accuracy, label='Train')
plt.plot(iterations, testing_accuracy, label='Test')
plt.ylabel('Accuracy')
plt.xlabel('iterations')
plt.legend(loc='upper right')
plt.show()